# Importing libraries

In [1]:
import sys           
import pandas as pd  
import sqlite3       

print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
- pandas 1.4.4
- sqlite3 3.39.3


# Connecting to the database

In [2]:
conn = sqlite3.connect('prod_sample.db')
conn

## Listing all the tables in the database.

In [3]:
query = 'SELECT name FROM sqlite_master where type= "table" '

pd.read_sql_query(query, conn)

,name
0,stock_description
1,online_retail_history


# Reading data from the `online_retail_history` table

In [4]:
query = 'SELECT * FROM online_retail_history'

onRetail = pd.read_sql(query, conn)

onRetail.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [5]:
onRetail.shape

(15321, 8)

# Read data from the `stock_description` table

In [6]:
query = 'SELECT * FROM stock_description'

stock = pd.read_sql(query, conn)

stock.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [7]:
stock.shape

(3952, 2)

# Aggregating the `online_retail_history` and `stock_description` tables

In [8]:
query = 'SELECT online_retail_history.*, stock_description.Description FROM online_retail_history LEFT JOIN stock_description ON online_retail_history.StockCode = stock_description.StockCode'

stockDes = pd.read_sql(query,conn)

stockDes.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [9]:
stockDes.shape

(17032, 9)

In [10]:
stockDes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17032 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      17032 non-null  object 
 1   StockCode    17032 non-null  object 
 2   Quantity     17032 non-null  int64  
 3   InvoiceDate  17032 non-null  object 
 4   Price        17019 non-null  float64
 5   CustomerID   13976 non-null  object 
 6   Country      17032 non-null  object 
 7   TotalAmount  17019 non-null  float64
 8   Description  17032 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


# Identifying and fixing corrupt data

In [11]:
stockDrp = stockDes.drop(['Invoice', 'CustomerID', 'Country'], axis = 'columns')
stockDrp.sample(7)

,StockCode,Quantity,InvoiceDate,Price,TotalAmount,Description
11355,20727,10,2011-08-23 14:55:00,1.65,16.50,LUNCH BAG BLACK SKULL.
12880,22720,9,2011-09-25 14:53:00,4.95,44.55,SET OF 3 CAKE TINS PANTRY DESIGN
3450,22197,100,2011-03-03 10:55:00,0.72,72.00,POPCORN HOLDER
14380,21212,3,2011-10-27 16:23:00,0.55,1.65,PACK OF 72 RETROSPOT CAKE CASES
8855,20727,1,2011-06-24 15:51:00,4.13,4.13,LUNCH BAG BLACK SKULL.
9505,85123A,32,2011-07-12 12:24:00,2.55,81.60,CREAM HANGING HEART T-LIGHT HOLDER
5228,85099B,10,2011-04-08 11:50:00,1.95,19.50,JUMBO BAG RED RETROSPOT


In [12]:
stockDes['Description'].value_counts()

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [13]:
# Removing rows where "Description" is just a question mark (?).
stockDrp = stockDrp[stockDrp.Description != '?']

stockDrp.sample(8)

,StockCode,Quantity,InvoiceDate,Price,TotalAmount,Description
9868,47566,60,2011-07-19 15:23:00,4.15,249.00,PARTY BUNTING
6013,84879,8,2011-04-26 11:59:00,1.69,13.52,ASSORTED COLOUR BIRD ORNAMENT
3738,22720,1,2011-03-09 14:30:00,4.95,4.95,SET OF 3 CAKE TINS PANTRY DESIGN
10444,85123A,32,2011-08-02 12:59:00,2.55,81.60,CREAM HANGING HEART T-LIGHT HOLDER
5316,20727,10,2011-04-11 10:01:00,1.65,16.50,LUNCH BAG BLACK SKULL.
6243,47566,1,2011-05-03 16:06:00,10.79,10.79,PARTY BUNTING
6604,20727,10,2011-05-09 14:44:00,1.65,16.50,LUNCH BAG BLACK SKULL.
1734,21212,1,2011-01-17 17:48:00,1.25,1.25,PACK OF 72 RETROSPOT CAKE CASES


# Identifying and removing duplicates

In [14]:
dups = stockDrp[stockDrp.duplicated()]

dups

,StockCode,Quantity,InvoiceDate,Price,TotalAmount,Description
178,20727,1,2010-12-03 11:19:00,1.65,1.65,LUNCH BAG BLACK SKULL.
497,84879,8,2010-12-08 12:46:00,1.69,13.52,ASSORTED COLOUR BIRD ORNAMENT
571,20725,1,2010-12-09 11:28:00,1.65,1.65,LUNCH BAG RED RETROSPOT
811,84879,8,2010-12-14 12:30:00,1.69,13.52,ASSORTED COLOUR BIRD ORNAMENT
935,22423,16,2010-12-16 10:08:00,10.95,175.20,REGENCY CAKESTAND 3 TIER
...,...,...,...,...,...,...
16542,20727,1,2011-12-01 12:53:00,1.65,1.65,LUNCH BAG BLACK SKULL.
16671,20727,2,2011-12-04 12:32:00,1.65,3.30,LUNCH BAG BLACK SKULL.
16673,20727,1,2011-12-04 12:32:00,1.65,1.65,LUNCH BAG BLACK SKULL.
16716,20727,1,2011-12-05 11:49:00,1.65,1.65,LUNCH BAG BLACK SKULL.


In [15]:
stockFin = stockDrp[~stockDrp.duplicated()]

stockFin.head()

,StockCode,Quantity,InvoiceDate,Price,TotalAmount,Description
0,85123A,6,2010-12-01 08:26:00,2.55,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,84879,32,2010-12-01 08:34:00,1.69,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,85123A,6,2010-12-01 09:02:00,2.55,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,85123A,6,2010-12-01 09:32:00,2.55,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,20725,10,2010-12-01 09:37:00,1.65,16.50,LUNCH BAG RED RETROSPOT


# Changing date formats

In [16]:
stockFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15167 entries, 0 to 17031
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StockCode    15167 non-null  object 
 1   Quantity     15167 non-null  int64  
 2   InvoiceDate  15167 non-null  object 
 3   Price        15155 non-null  float64
 4   TotalAmount  15155 non-null  float64
 5   Description  15167 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 829.4+ KB


In [17]:
stockFin['InvoiceDate'] = pd.to_datetime(stockFin['InvoiceDate'], format = '%Y-%m-%d')

C:\Users\MSI1\AppData\Local\Temp\ipykernel_14872\1043688626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stockFin['InvoiceDate'] = pd.to_datetime(stockFin['InvoiceDate'], format = '%Y-%m-%d')


In [18]:
stockFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15167 entries, 0 to 17031
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   StockCode    15167 non-null  object        
 1   Quantity     15167 non-null  int64         
 2   InvoiceDate  15167 non-null  datetime64[ns]
 3   Price        15155 non-null  float64       
 4   TotalAmount  15155 non-null  float64       
 5   Description  15167 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 829.4+ KB


In [19]:
stockFin.sample(7)

,StockCode,Quantity,InvoiceDate,Price,TotalAmount,Description
6438,20725,20,2011-05-06 13:52:00,1.65,33.00,LUNCH BAG RED RETROSPOT
2785,22720,3,2011-02-15 15:51:00,4.95,14.85,SET OF 3 CAKE TINS PANTRY DESIGN
13963,47566,1,2011-10-17 15:06:00,4.95,4.95,PARTY BUNTING
16078,85099B,20,2011-11-24 10:39:00,2.08,41.60,JUMBO BAG RED RETROSPOT
13126,20725,1,2011-09-29 16:48:00,1.65,1.65,LUNCH BAG RED RETROSPOT
3790,85099B,1,2011-03-10 15:01:00,1.95,1.95,JUMBO BAG RED RETROSPOT
8922,22423,2,2011-06-28 08:19:00,12.75,25.50,REGENCY CAKESTAND 3 TIER


# Loading the dataset into a pickle file

In [20]:
stockFin.to_pickle('online_history_cleaned.pickle')

## Closing the Database Connection

In [21]:
conn.close()